<a href="https://colab.research.google.com/github/ThiagSampaio/DeepLearning/blob/main/Regress%C3%A3o_Log%C3%ADstica_com_Rede_Neural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regressão Logística com Rede Neural

Vamos construir um classificador de regressão logística para reconhecer gatos.

** O que será mostrado: **
- Construir uma arquitetura geral de um algoritmo de aprendizado, incluindo:
    - Inicializando os parâmetros
    - Calculando a função custo e seu gradiente
    - Usando um algoritmo de otimização(gradiente descente)
- Pegar as três funções acima e aplicá-las em uma função modelo principal, na ordem correta.

## Tabela de Conteúdos
- [1 - Pacotes](#1)
- [2 - Olhando o problema](#2)
    - [Demonstração 1](#ex-1)
    - [Demonstração 2](#ex-2)
- [3 - Arquitetura geral de um algoritmo de aprendizado](#3)
- [4 - Construindo as partes do nosso algoritmo](#4)
    - [4.1 - Funções úteis](#4-1)
        - [Demonstração 3 - sigmoid](#ex-3)
    - [4.2 - Inicializando parâmetros](#4-2)
        - [Demonstração 4 - inicializando_com_zeros](#ex-4)
    - [4.3 - Propagação direta e inversa](#4-3)
        - [Demonstração 5 - propagação](#ex-5)
    - [4.4 - Optimization](#4-4)
        - [Demonstração 6 - otimização](#ex-6)
        - [Demonstração 7 - predição](#ex-7)
- [5 - Aglutinar todas as funções em um modelo](#5)
    - [Demonstração 8 - modelo](#ex-8)
- [6 - Análises](#6)
- [7 - Testando com sua prória imagem](#7)



<a name='1'></a>
## 1 - Packages ##

Primeiro, vamos rodar a célula abaixo para importar todos os pacotes que usaremos durante essa demonstração.
- [numpy](https://numpy.org/doc/1.20/) é um pacote fundamental para computação científica com python
- [h5py](http://www.h5py.org) é um pacote comum para interagir com arquivos armazenados como .h5
- [matplotlib](http://matplotlib.org) é uma famosa biblioteca para plotar gráficos.
- [PIL](https://pillow.readthedocs.io/en/stable/) and [scipy](https://www.scipy.org/) são usados para testar o modelo com sua própria figura no final do estudo. 

In [14]:
import numpy as np
import copy
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage
from lr_utils  import load_dataset
from public_tests import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

<a name='2'></a>
## 2 - Olhando para o problema##

**Declaração do Problema**: Foi dado a você um dataset("data.h5") contendo:

    - um conjunto de treinamento de imagens m_train rotuladas como gato (y = 1) ou não gato (y = 0)
    - um conjunto de teste de imagens m_test rotuladas como gato ou não gato
    - cada imagem tem forma (num_px, num_px, 3) onde 3 é para os 3 canais (RGB). Assim, cada imagem é quadrada (altura = num_px) e (largura = num_px).

Construiremos um algoritmo de reconhecimento de imagem simples que pode classificar corretamente as imagens como gatos ou não.

Vamos nos familiarizar mais com o conjunto de dados. Carregue os dados executando o código a seguir.

In [ ]:
<a name='2'></a>
## 2 - Overview of the Problem set ##

**Problem Statement**: You are given a dataset ("data.h5") containing:
    - a training set of m_train images labeled as cat (y=1) or non-cat (y=0)
    - a test set of m_test images labeled as cat or non-cat
    - each image is of shape (num_px, num_px, 3) where 3 is for the 3 channels (RGB). Thus, each image is square (height = num_px) and (width = num_px).

You will build a simple image-recognition algorithm that can correctly classify pictures as cat or non-cat.

Let's get more familiar with the dataset. Load the data by running the following code.